In [8]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
import torch
from IPython.display import display_markdown
print(torch.cuda.get_device_name(0))
torch.cuda.empty_cache()
vram = torch.cuda.get_device_properties(0).total_memory / 1e9
print(f"Total VRAM: {vram} GB")

NVIDIA GeForce RTX 4080 SUPER
Total VRAM: 17.170956288 GB


In [1]:
# !pip install faiss-cpu # for vector db
# !pip install PyPDF2 # for parsing pdfs
# !pip install pandas
# !pip install scikit-learn  # For TfidfVectorizer, cosine distance
# !pip install langchain # for text splitting
# !pip install tqdm # for progress bar
# !pip install numpy

from typing import Dict, Any, List, Tuple
from helpers import (
    DocumentProcessor,
    VectorStore,
    HybridSearcher,
    Document,
    get_completion,
    get_embeddings_batch
)
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

2025-02-25 23:29:51,122 - INFO - Loading tokenizer and models...
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
2025-02-25 23:29:52,329 - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
2025-02-25 23:29:58,815 - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2025-02-25 23:30:04,396 - WARNING - Some parameters are on the meta device because they were offloaded to the cpu.
2025-02-25 23:30:04,398 - INFO - Quantized model loaded successfully.


In [2]:
def load_documents(file_path):
    # Load and chunk document
    text = DocumentProcessor.load_pdf(file_path)
    documents = DocumentProcessor.chunk_text(text)
    
    # Create and return vector store
    vector_store = VectorStore()  # HybridSearcher()
    vector_store.create_index(documents, force_recreate=True)
    print(f"Processed {len(documents)} chunks")
    
    return vector_store

vector_store = load_documents("SS_Bene.pdf")

Loading PDF pages:   0%|          | 0/32 [00:00<?, ?it/s]

Cleaning extracted text...


Processing chunks:   0%|          | 0/46 [00:00<?, ?chunk/s]

Final number of chunks: 46
Creating new index...


Generating embeddings:   0%|          | 0/46 [00:00<?, ?it/s]

Saved index with 46 documents
Processed 46 chunks


In [3]:
query = "What is Social Security?"
results = vector_store.search(query, k=5)
results

[(Document(content='=== Page 1 === 2025 Understanding the Benefits SSA.gov === Page 2 === === Page 3 === What’s inside Social Security: a simple concept 1 What you need to know about Social Security while you’re working 4 What you need to know about benefits 7Benefits for your family 11 When you’re ready to apply for benefits 14Supplemental Security Income (SSI) program 16 Right to appeal 17 Online personal my Social Security account 17Medicare 18Some facts about Social Security 22 Contacting Us 23 === Page 4 === === Page 5 === 1Social Security: a simple concept Social Security reaches almost every family, and at some point, touches the lives of nearly all Americans. We help older Americans, workers with disabilities, and families in which a spouse or parent dies. We estimate that about 184 million people worked in Social Security-covered employment in 2024 and paid Social Security taxes. As of September 2024, about 68 million people received monthly Social Security benefits', metadata

In [ ]:
def query(question: str, vector_store, top_k: int = 5) -> str:
    """
    Process a question and return an answer with sources.

    Args:
        question: User's question
        vector_store: Indexed document store
        top_k: Number of relevant chunks to retrieve

    Returns:
        Answer string
    """
    # Get relevant chunks
    relevant_chunks = vector_store.search(question, k=top_k)
    contents = "\n".join([x[0].content for x in relevant_chunks])

    prompt = f"""
    You are a helpful assistant specialized in extracting information from documents.

    I have the following question:
    "{question}"

    Here are the relevant document contents:
    {contents}

    Please provide a concise and accurate answer to the question based on the contents above.
    Only use the information present in the contents. If no relevant information is found, respond with "No".

    Answer:
    """

    # Call the completion function
    response = get_completion(prompt)
    print(response)

    # Clear GPU memory and force context termination
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    del prompt, contents, relevant_chunks  # Clear variables from memory

    return response  # Explicitly return the result to end the function



query("How much will your survivors get?")


AttributeError: 'list' object has no attribute 'search'

In [ ]:
def get_relevant_chunks(
    question,
    vector_store,
    top_k = 3):
    """
    Retrieve relevant document chunks for a question
    Args:
        question: User's question
        vector_store: Indexed document store
        top_k: Number of chunks to retrieve
    Returns:
        List of (document, relevance_score) tuples
    """
    return vector_store.search(question, k=top_k)

def generate_answer(
    question,
    relevant_chunks):
    """
    Generate an answer using retrieved chunks
    Args:
        question: User's question
        relevant_chunks: List of relevant documents and their scores
    Returns:
        Generated answer
    """
    # Format context from relevant chunks
    contexts = [
        f"[{i}] {doc.content}"
        for i, (doc, _) in enumerate(relevant_chunks, 1)
    ]
    
    prompt = f"""You are a helpful assistant.
    Answer the question based on the context. If the answer isn't in the context, say so.

    Question: {question}

    Context:
    {"/n/n".join(contexts)}

    Answer:"""
    
    return get_completion(prompt)

def query(
    question,
    vector_store,
    top_k = 10):
    """
    Process a question and return an answer with sources
    Args:
        question: User's question
        vector_store: Indexed document store
        top_k: Number of relevant chunks to retrieve
    Returns:
        Dictionary containing answer and sources
    """
    # Get relevant chunks
    relevant_chunks = get_relevant_chunks(question, vector_store, top_k)
    
    # Generate answer
    answer = generate_answer(question, relevant_chunks)
    
    # Format and return response
    return answer, relevant_chunks

question = "How does Wells Fargos benefits compare to Vanguard?"
answer, relevant_chunks = query(question, vector_store, top_k=10)

print("\nAnswer:", answer)

# Print sources
print("\nSources:")
for i, source in enumerate(relevant_chunks, 1):
    print(f"\n[{i}] Relevance score: {relevant_chunks[i-1][1]:.2f}")
    print(f"Preview: {relevant_chunks[i-1][0].content}")


KeyboardInterrupt: 